In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, Dropout,MaxPooling2D

np.random.seed(123)  # <1>

X = np.load('../features-40k.npy')  # <2>
Y = np.load('../labels-40k.npy')
samples = X.shape[0]
size = 9
input_shape = (size,size,1)

print(X.shape)
#
X = X.reshape(samples,size,size,1)
print(X.shape)
train_samples = int(0.9*samples)
X_train, X_test = X[:train_samples], X[train_samples:]
Y_train, Y_test = Y[:train_samples], Y[train_samples:]



Using TensorFlow backend.


(41439, 1, 9, 9)
(41439, 9, 9, 1)


In [2]:
model = Sequential()
model.add(Conv2D(filters=48,
                   kernel_size=(3,3),
                   activation='relu',
                   padding = 'same',
                   input_shape=input_shape))
model.add(Dropout(rate=0.5))

model.add(Conv2D(48,(3,3),padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(rate=0.5))

model.add(Flatten())
model.add(Dense(512, activation='relu' ))
model.add(Dropout(rate=0.5)
)
model.add(Dense(size*size, activation='softmax'))
model.summary() 

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 9, 9, 48)          480       
_________________________________________________________________
dropout_1 (Dropout)          (None, 9, 9, 48)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 9, 9, 48)          20784     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4, 4, 48)          0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 4, 4, 48)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 768)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)              

2022-01-25 16:03:16.443718: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2022-01-25 16:03:16.444277: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter op setting: 16. Tune using inter_op_parallelism_threads for best performance.


In [3]:
model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

In [4]:
model.fit(X_train, Y_train,
          batch_size=64,
          epochs=10,
          verbose=1,
          validation_data=(X_test, Y_test))

score = model.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 37295 samples, validate on 4144 samples
Epoch 1/10
37295/37295 [==============================] - 16s 424us/step - loss: 4.3706 - accuracy: 0.0188 - val_loss: 4.3198 - val_accuracy: 0.0285
Epoch 2/10
37295/37295 [==============================] - 17s 452us/step - loss: 4.3141 - accuracy: 0.0230 - val_loss: 4.2763 - val_accuracy: 0.0268
Epoch 3/10
37295/37295 [==============================] - 18s 475us/step - loss: 4.2881 - accuracy: 0.0236 - val_loss: 4.2577 - val_accuracy: 0.0253
Epoch 4/10
37295/37295 [==============================] - 19s 517us/step - loss: 4.2709 - accuracy: 0.0230 - val_loss: 4.2447 - val_accuracy: 0.0256
Epoch 5/10
37295/37295 [==============================] - 26s 705us/step - loss: 4.2566 - accuracy: 0.0234 - val_loss: 4.2303 - val_accuracy: 0.0275
Epoch 6/10
37295/37295 [==============================] - 22s 594us/step - loss: 4.2375 - accuracy: 0.0233 - val_loss: 4.2116 - val_accuracy: 0.0278
Epoch 7/10
37295/37295 [==============================] -

In [6]:
samples

41439

In [11]:
def softmax(x):
    e_x= np.exp(x)
    e_x_sum = np.sum(e_x)
    return e_x/e_x_sum

x= np.array([100,100,20,0,100,-33])
print(softmax(x))

[3.33333333e-01 3.33333333e-01 6.01617129e-36 1.24002533e-44
 3.33333333e-01 5.77713681e-59]


In [5]:
test_board = np.array([[0,0,0,0,0,0,0,0,0,
                       0,0,0,0,0,0,0,0,0,
                       0,0,1,-1,1,-1,0,0,0,
                       0,0,1,-1,1,-1,0,0,0,
                       0,0,0,1,-1,0,0,0,0,
                       0,0,0,0,0,0,0,0,0,
                       0,0,0,0,0,0,0,0,0,
                       0,0,0,0,0,0,0,0,0,
                       0,0,0,0,0,0,0,0,0]])

In [12]:
test_board = test_board.reshape(1,9,9,1)

In [13]:
move_probs = model.predict(test_board)[0]
i=0
for row in range(9):
    row_formatted = []
    for col in range(9):
        row_formatted.append('{:.3f}'.format(move_probs[i]))
        i += 1
    print(' '.join(row_formatted))

0.001 0.002 0.003 0.003 0.004 0.003 0.002 0.001 0.001
0.002 0.007 0.010 0.015 0.018 0.014 0.009 0.005 0.002
0.002 0.010 0.004 0.029 0.024 0.029 0.005 0.006 0.002
0.003 0.014 0.026 0.043 0.030 0.023 0.014 0.016 0.003
0.003 0.018 0.038 0.031 0.038 0.029 0.029 0.015 0.004
0.003 0.014 0.029 0.042 0.042 0.025 0.027 0.011 0.003
0.002 0.009 0.004 0.029 0.030 0.030 0.004 0.008 0.002
0.002 0.006 0.008 0.014 0.017 0.015 0.009 0.005 0.002
0.001 0.002 0.002 0.003 0.003 0.003 0.002 0.002 0.001
